In [ ]:
#installation de facebook scrapper
!pip install facebook-scraper

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
from parsel import Selector
from selenium.webdriver.common.keys import Keys
from facebook_scraper import get_posts
from multiprocessing import Pool, cpu_count
import re
from requests import get
import pandas as pd
import numpy as np
import time
import os
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from fake_useragent import UserAgent


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
rechercheGoogle("yopougon",1)

In [4]:
def headless(proxy: str = "") :
        ua = UserAgent()
        userAgent = ua.random
        options = webdriver.ChromeOptions()
        options.add_argument("headless")
        options.add_argument("window-size=1500,1200")
        options.add_argument("no-sandbox")
        options.add_argument("disable-dev-shm-usage")
        options.add_argument("disable-gpu")
        options.add_argument("log-level=3")
        options.add_argument(f"user-agent={userAgent}")

        if proxy != "":
            proxy = True
            options.add_argument("proxy-server={}".format(proxy))

        driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)
        return driver    


In [5]:
# initialisation de selemium
def rechercheGoogle(localite,nombrePage):
    # chargement de webdriver et du navigateur
    driver = headless()

    # aller sur facebook
    driver.get('http://www.google.com')
    # recherche le champs de saisi
    search_query = driver.find_element_by_name('q')
    search_query.send_keys('site:facebook.com AND'+'"'+localite +'"AND "'+ "maison" +'"')
    search_query.send_keys(Keys.RETURN)
    test=True
    n=0
    lienGroup=[]
    while test:
        if n<nombrePage:
            try:
            ####### recuperer les liens des differents groupes
                facebook_urls = driver.find_elements_by_xpath("//*[@class='yuRUbf']/a")
                for url in facebook_urls:
                    if "facebook" in url.get_attribute("href"):
                        for id in range(len(url.get_attribute("href").split("/"))):
                            if url.get_attribute("href").split("/")[id]=="groups":
                                lienGroup.append(url.get_attribute("href").split("/")[id+1])
                                break    
                suivantpage = driver.find_element_by_id('pnnext')
                sleep(5)
                suivantpage.click()
                n=n+1
            except:
                    test=False
                    continue
        else:
                test=False

    return lienGroup

            

In [6]:
def postGroup(nbpage,idGroup):
    n=0
    ListPost=[]
    for post in get_posts(idGroup, cookies='cookies.txt', extra_info=True, pages=nbpage, options={"comments": False}):
        print(post)
        ListPost.append(post)
ua = UserAgent()

In [ ]:
ListIdGroup=rechercheGoogle("yopougon",1)
nbpage=1
items=[(nbpage,x) for x in ListIdGroup]

In [ ]:
def multiprocessing(postGroup , list):
    with Pool(cpu_count()) as p:
        print(cpu_count())
        rec=p.starmap( postGroup,list)    
        p.terminate()
        p.join()
        return rec
# Lanceur du script
if __name__ == '__main__':
        start_time = time.time()
        list = items
        ListeDictionnaire=multiprocessing(postGroup , list)
        # decomenter la ligne suivante pour afficher les valeurs
        #print(ListeDictionnaire)
        #pd.DataFrame(ListeDictionnaire).to_csv("output.csv")
        end_time = time.time() - start_time
        # Fin des traitement
        print("Temps total de compilation "+str(end_time))
      

In [ ]:
out=postGroup(1,"1977749725832531")

In [2]:
import pandas as pd
import numpy as np
bd = pd.read_csv("data.csv")
bd.head(5)

,Unnamed: 0,post_text
0,0,‟ Yannick Blehi s’est vu être dépouillé de son...
1,1,3pieces 170000f Yopougon batim extension ou Vi...
2,2,Bonjour besoin urgent d'un studio Dee 50000 au...
3,3,SONGON GRAVIER\nREZ-DE-CHAUSSÉE\n\n✅2 PIÈCES\n...
4,4,studio à yopougon cité verte bon standing avec...


In [3]:
def typeMaison(publication,bag_app,bag_mais):
     dicoType={}
     count=1
     for word in publication:
        if word in bag_app:
            dicoType[count]='appartement'
            count=count+1  
        elif word in bag_mais:
            dicoType[count]='maison Individuel'
            count=count+1  
     return dicoType

In [78]:

def check(post_elmt):
    post_elmt.replace(".","")
    post_elmt = re.findall("\d+", post_elmt)
    if len(post_elmt) != 0 :
        post_elmt = post_elmt[0]  
        return post_elmt
    return 0

def montant_loyer_piece(sac_mot, post):
    dico_montant = {}
    post_suiv = ""
    cpt=0
    for i in range(0,len(post)):
        loyer=0
        if post[i] in sac_mot:
            # break
            post_pred = post[i-1]
            if i != (len(post)-1):
                post_suiv = post[i+1]
            loyer = check(post_pred)
            if loyer!=0:
                    cpt+=1
                    dico_montant[cpt]= loyer
        
            if ((loyer == 0) and (i != (len(post)-2))):
                loyer = check(post[i+1])
                if loyer!=0:
                    cpt+=1
                    dico_montant[cpt]= loyer
        
        if loyer==0:
            val=re.findall("[a-zA-Z]+", post[i])
            if len(val)!=0:
                if val[0].upper() in sac_mot:
                    loyer= check(post[i])
                    if loyer!=0:
                        cpt=cpt+1
                        dico_montant[cpt]= loyer

    return dico_montant


In [6]:
def standing(sac,eco,haut,moyen):
    dicoStanding = dict()
    k = 1
    for i in sac:
        if i in eco:
            dicoStanding[k] = "econonmique"
            k += 1
            continue
        if i in haut:
            dicoStanding[k] = "haut"
            k += 1
            continue
        if i in moyen:
            dicoStanding[k] = "moyen"
            k += 1     
    return dicoStanding

In [ ]:
immeuble_sac_mot =['APPARTEMENT','APPARTEMENTS','APARTEMENT','APARTEMENTS', 
           'APPATEMENT','APPATEMENTS', 'APPAT', 'APPATS','IMMEUBLE',
           'IMMEUBLES','IMEUBLE','IMMEBLE',  'IMMEBLES','IMEBLE', 'IMEBLES','1ER','2EM','ETAGE']
villa_sac_mot=["VILLA","VILA",'VILLAS','VILAS']
piece_sac_mot=["PIECE","PIECES","PIEC",'PCS']
montant_loyer_sac_mot = [ "FCFA","F","CFA","FR","MIL","MILLE","MILLES",'XOF','FA','€','$']
motCleLocationValide=["LOYER","LOUER","LOUE","LOU","LOCATION","LOCASSION","LOCASION","VISITE","CAUTION","CAUSION","CAUSSION","COSION","COSSION"]
motCleLocationExclu=["VENTE","VENT","TERRAIN","TERRIN","TERAIN","FONCIER","ACHETER","ACHETE"]

In [ ]:
from unidecode import unidecode
import re
def gnb_piece(texte):

    compltexte = texte

    compltexte = compltexte.replace("*", "")
    compltexte = compltexte.replace(",", "")
    compltexte = compltexte.replace(".", "")
    

    compltexte = compltexte.split() 
    compltexte = [mot.upper() for mot in compltexte]
    compltexte = [unidecode(mot) for mot in compltexte]
    


    motif = ["PIECE", "PIECES","PCS","PIECE,","PIEGE","PIEGES"]

    for i in range(1,len(compltexte)):
        if compltexte[i] in motif:
            position = i
            # break
            nb_pieces = compltexte[i-1]
            nb_pieces = nb_pieces.replace(" ", "")
            nb_pieces = re.findall("\d+", nb_pieces)
            nb_pieces = int(nb_pieces[0])
            return nb_pieces


In [ ]:

def first(listTraite):
    listTraite=listTraite.strip().replace("\n"," ")
    val=[]
    txtList = listTraite.split(" ")
    listTraite = [ str(unidecode.unidecode(x).upper()) for x in txtList]
    if "YOPOUGON"  in listTraite or "YOP" in listTraite:
        for mot in listTraite:
            if mot in motCleLocationValide:
                val= listTraite                
            elif mot in motCleLocationExclu:
                val=  []
            else:
                val= listTraite
    return val
publist=[]
for pub in bd["post_text"]:
  try:
    rt=first(pub)
    if len(rt) !=0:
        publist.append(rt)

  except:
        continue
        

In [ ]:
for publ in publist:
    typeM= typeMaison(publ,immeuble_sac_mot,villa_sac_mot)
    print("#####################")
    print(typeM)
    piece = montant_loyer_piece(piece_sac_mot,publ)
    print("#####################")
    print(piece)
    montant = montant_loyer_piece(montant_loyer_sac_mot,publ)
    print("#####################")
    print(montant)
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")


# fonction ON EST SUR
## considerons qu'un post est une offre on effectue le traitement pour recuperer les informations 
## On fera ensuite un decompte des parametres obtenu afin de confirmer s'il s'agit d'une offre ou pas

# ALGO

### 1. recherche du type (verifier si le mot est pas vide toujours)
#### A. si au moins un mot fait parti du sac a mot  ["APPARTEMENT","APARTEMENT","IMEUBLE","IMMEUBLE"] on au un IMMEUBLE ou si le mot element de  ["MAISON","MAISSON","VILLA","VILA"]  on a maison ind
#### B. On aura un dictionniare  du type {"ordre","valeur"}
#### C. Si on a rien la liste aura un dictionnaire vide

### 2.  Fonction standing (verifier si le mot est pas vide toujours)
#### A. si au moins un mot fait parti du sac a mot  ["",""] on au  HAUT , si le mot element de  ["","","",""]  on a Economique ou si mot dans ["",""] moyen
#### B. On aura un dictionniare  du type {"ordre","valeur"
#### C. Si on a rien la liste aura un dictionnaire vide

### 3.  montant_loyer (verifier si le mot est pas vide toujours)
#### A. si le mot est element du sac a mot [] definit
##### A.1 on test le mot precedent
###### A.1.1 s'il fait partie du sac a mot franc (fr,mil,mille.....) on converti  le precedent en integer si ca passe on a le montant
###### A.1.2 s'il fait pas partie du sac a mot on converti en integer si ok on a le montant
##### A.2 si A.1 marche pas sur la liste de texte on applique le A.1.1 et A.1.2 mais la au suivant
##### B. on empile les valeurs dans un dictionaire {"ordre","valeur"}
#### C. Si on a rien la liste aura un dictionnaire vide

### 4. NB_PIECE (verifier si le mot est pas vide toujours)
#### A.Meme procedure que le 3 mais la on s'assure que la valeur converti est element de [0-9]




# Fonction de Decision
## 1-cas
### Tous dictionnaires sont vide ou au moins un seul non vide
#### Le post est pas valable

## 2-cas
### Au moins deux dictionnaires non vides mais de taille 1
#### le post equivaut a une offre

## 3-cas
#### Au moins deux dictionnaires non vides mais de taille N
#### le post equivaut a N offre

## On Enregistre les valeurs dans un data en suivant l'ordre



In [7]:
import unidecode

In [ ]:
def nettoyage():
    

txt.strip()

In [67]:
piece

{}

In [47]:
for i in range(21):
    print(out[i]['post_text'])
    print("#######################################################################################"+str(i)+"#######################################################################################")

Bonjour famille , ici dans ce groupe désormais seul les postes qui portent sur la Commune de Cocody et Koumassi , Bingerville seront approuver 🙏

Merci de le respecter
#######################################################################################0#######################################################################################
🤩🤩🤩OPPOTUNITE DE VENTE DE BIENS IMMOBILIER🤩🤩🤩
👉 Angré Chu djorogobité 500 M2 50 millions à débattre titre foncier effectif ACD en cours
👉Deux terrains 435 M2 et 600 M2 30 millions et 25 millions. BingervillePlus palais de justice
👉Appartement de 3 pièces en vente a Angré extension dernière 34em au premier étage avec balcon a 30 millions ko
👉ANGRÉ NOUVEAU CHU secteur DJOROGOBITE-1. Vente villa basse 4 pièces avec garage 1 voiture sur 200 m2 avec ACD au prix de 42 millions.
Infos : 07 47 82 56 02 / 01 01 79 79 09
#######################################################################################1###################################################